In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time

rng = np.random.default_rng()

In [ ]:
def ordered_state(width):
    return np.exp(2j * np.pi * np.zeros((width,width,width,width,4)))

def disordered_state(width):
    return np.exp(2j * np.pi * np.random.rand(width,width,width,width,4))

In [ ]:
def run_lattice_heatbath(state,beta,n):
    """Perform n heatbath updates on the lattice state."""
    for _ in range(n):
        lattice_heatbath_update(state,beta)

def lattice_heatbath_update(state,beta):
    """Perform a heatbath update on a random link in the lattice state."""
    width = len(state)
    link_index = random_link_index(width)
    state[link_index] = sample_link_variable(state,beta,link_index)

def random_link_index(width):
    """Return a random link index (n_x,n_y,n_z,kappa)."""
    return tuple(rng.integers(0,[width,width,width,width,4]))

def sample_link_variable(state,beta,link_index):
    """Sample link variable U = exp(i*theta)."""
    link_variable_sum = relevant_link_variable_sum(state,link_index)
    alpha = beta*np.real(link_variable_sum)
    phi = np.angle(link_variable_sum)
    while True:
        Z = rng.uniform(0,1)
        x = -1 + np.log(1 + (np.exp(2*alpha) - 1)*Z)/alpha

        Q = np.exp(alpha*(np.cos(np.pi/2*(1-x))-x))
        Q_max = np.exp(0.2105137*alpha)

        Z_prime = rng.uniform(0,1)
        if Q/Q_max > Z_prime:
            theta = np.pi*(1-x)/2 - phi
            return np.exp(1j*theta)

def relevant_link_variable_sum(state,link_index):
    """Return the sum of the link variables present in the plaquettes containing the relevant link,
    without the contribution of the link itself."""
    width = len(state)
    n = np.array(link_index[:4])
    kappa = link_index[-1]
    kappa_hat = np.array(get_unit_vector(kappa))

    link_variable_sum = 0
    for nu in range(4):
        if nu != kappa:
            nu_hat = get_unit_vector(nu)
            contribution = state[get_lattice_vector(n+kappa_hat,width)][nu]
            contribution *= state[get_lattice_vector(n+nu_hat,width)][kappa]
            contribution *= state[get_lattice_vector(n,width)][nu]
            link_variable_sum += contribution

    return link_variable_sum

def get_unit_vector(index):
    """Return the unit vector from the dimension index."""
    vector = np.zeros(4).astype(int)
    vector[index] = 1
    return vector

def get_lattice_vector(vector,width):
    """Get the lattice vector periodic with the lattice size."""
    return tuple(map(lambda i: i % width, vector))

In [ ]:
def average_plaquette_action(state):
    """Compute action of the lattice state."""
    width = len(state)

    action = 0
    for n in lattice_vertices(width):
        for mu in range(4):
            for nu in range(4):
                if mu < nu:
                    mu_hat, nu_hat = get_unit_vector(mu), get_unit_vector(nu)
                    contribution = state[get_lattice_vector(n,width)][mu]
                    contribution *= state[get_lattice_vector(n+mu_hat,width)][nu]
                    contribution *= state[get_lattice_vector(n+nu_hat,width)][mu]
                    contribution *= state[get_lattice_vector(n,width)][nu]
                    action += (1 - np.real(contribution))

    num_plaquettes = width*width*width*width*6
    return action / num_plaquettes

def lattice_vertices(width):
    """Compute the vertices of the lattice with given width."""
    vertices = []
    for n_x in range(width):
        for n_y in range(width):
            for n_z in range(width):
                for n_t in range(width):
                    vertex = (n_x,n_y,n_z,n_t)
                    vertices.append(vertex)
    return vertices

In [ ]:
def gather_data(state,beta,k,n,measurements):
    filename = create_dataset(state,beta,k,n)
    # actions = run_simulation(state,beta,k,n,measurements)

def create_dataset(state,beta,k,n):
    width = len(state)
    filename = "data_w{}_b{}_{}.hdf5".format(width,beta,time.strftime("%Y%m%d%H%M%S"))

    # create file
    with h5py.File(filename,'a') as f:
        dataset = f.create_dataset("actions",(0,),maxshape=(None,),dtype='i4',chunks=True)

        # store metadata
        dataset.attrs["version"] = 1
        dataset.attrs["lattice size"] = len(state)
        dataset.attrs["inverse temperature"] = beta
        dataset.attrs["equilibration sweeps"] = k
        dataset.attrs["measurement sweeps"] = n
        dataset.attrs["start time"] = time.asctime()
        dataset.attrs["end time"] = 0

    return filename